In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split


In [ ]:
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
print(train.shape)
print(train.describe)
train.head()

In [ ]:
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
print(test.shape)
print(test.describe)
test.head()

In [ ]:
X_train = train.drop("label", axis=1).values

In [ ]:
Y_train = train['label'].values
Y_train

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_train /= 255

In [ ]:
X_train.shape

In [ ]:
X_test = test.values
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_test = X_test.astype('float32')
X_test /= 255
X_test.shape

In [ ]:
Y_train= to_categorical(Y_train)
num_classes = Y_train.shape[1]
num_classes

In [ ]:
input_shape = (28, 28, 1)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(256, kernel_size=(3,3), activation='relu', input_shape=input_shape, padding='Same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=input_shape, padding='Same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
model.summary()

In [ ]:
earlyStopping = keras.callbacks.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = keras.callbacks.callbacks.ModelCheckpoint('/kaggle/working/.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = keras.callbacks.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
history = model.fit(X_train, Y_train, batch_size=50, verbose=1, epochs=50, validation_data=(X_val, Y_val), callbacks=[earlyStopping, mcp_save, reduce_lr_loss])

In [ ]:
predictions = model.predict_classes(X_test, verbose=0)

submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})

In [ ]:
submissions.to_csv("output.csv", index=False)